## Perform imports and load the dataset

In [3]:
import numpy as np
import pandas as pd
!pip install nltk
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [18]:
df_train = pd.read_csv('https://raw.githubusercontent.com/SibusisoTL/classification-predict/master/train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/SibusisoTL/classification-predict/master/test.csv')

In [43]:
sample_submission = pd.read_csv('https://raw.githubusercontent.com/SibusisoTL/classification-predict/master/sample_submission.csv')

In [45]:
sample_submission.head()

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,1


In [19]:
df_train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [20]:
df_test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


In [21]:
print(len(df_train))

15819


In [22]:
print(len(df_test))

10546


In [23]:
df_train.isnull().sum()

sentiment    0
message      0
tweetid      0
dtype: int64

In [24]:
df_test.isnull().sum()

message    0
tweetid    0
dtype: int64

In [25]:
df_train['sentiment'].value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

In [26]:
# Check for whitespace strings (it's OK if there aren't any!):
blanks = []  # start with an empty list

for i,sntm,msg,twts in df_train.itertuples():  # iterate over the DataFrame
    if type(msg)==str:            # avoid NaN values
        if msg.isspace():         # test 'message' for whitespace
            blanks.append(i)     # add matching index numbers to the list
        
len(blanks)

0

In [27]:
df = df_train.copy()

In [28]:
#Split the train data into train and validation 

from sklearn.model_selection import train_test_split

X = df["message"]
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Build a pipeline to vectorize the date, then train and fit a model
Use LinearSVC..

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [30]:
# Form a prediction set
predictions = text_clf.predict(X_test)

In [31]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 198   50  157   23]
 [  28  319  319   71]
 [  38  125 2439  259]
 [   4   23  214  954]]


In [32]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

          -1       0.74      0.46      0.57       428
           0       0.62      0.43      0.51       737
           1       0.78      0.85      0.81      2861
           2       0.73      0.80      0.76      1195

    accuracy                           0.75      5221
   macro avg       0.72      0.64      0.66      5221
weighted avg       0.74      0.75      0.74      5221



In [33]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.748898678414097


## Combine Steps with TfidVectorizer

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(10598, 23895)

## Train a Classifier
Here we'll introduce an SVM classifier that's similar to SVC, called LinearSVC. LinearSVC handles sparse input better, and scales well to large numbers of samples.

In [36]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC()

## Build a Pipeline

In [37]:
from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train) 

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

## Test the classifier and display results

In [38]:
# Form a prediction set
predictions = text_clf.predict(X_test)

In [39]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 198   50  157   23]
 [  28  319  319   71]
 [  38  125 2439  259]
 [   4   23  214  954]]


In [40]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

          -1       0.74      0.46      0.57       428
           0       0.62      0.43      0.51       737
           1       0.78      0.85      0.81      2861
           2       0.73      0.80      0.76      1195

    accuracy                           0.75      5221
   macro avg       0.72      0.64      0.66      5221
weighted avg       0.74      0.75      0.74      5221



In [41]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.748898678414097


## Create csv to upload to Kaggle

In [52]:
#Make predictions using the features from the test data set
predictions = text_clf.predict(df_test['message'])
#predictions = clf.predict(df_test[])

#Display our predictions - they are either 0 or 1 for each training instance 
#depending on whether our algorithm believes the person survived or not.
predictions

array([1, 1, 1, ..., 2, 0, 1], dtype=int64)

In [53]:
#Create a  DataFrame with the tweetid and our prediction regarding whether it good or bad
submission = pd.DataFrame({'tweetid':df_test['tweetid'],'sentiment':predictions})

#Visualize the first 5 rows
submission.head()

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,0


In [54]:
#This is saved in the same directory as your notebook
filename = 'submission.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: submission.csv
